In [139]:
import pandas as pd
import os 

### **1. Carga de Datos**

In [162]:
df_estados_USA= pd.read_csv("D:/PROYECTO FINAL/Google Maps/estados_usa.csv",delimiter = ';', encoding = "utf-8")

In [2]:
rutaJson = "D:/PROYECTO FINAL/Google Maps/metadata-sitios/"     # Ruta de los json de Sitios
archivos_json = [os.path.join(rutaJson, archivo) for archivo in os.listdir(rutaJson) if archivo.endswith('.json')]
archivos_json

['D:/PROYECTO FINAL/Google Maps/metadata-sitios/1.json',
 'D:/PROYECTO FINAL/Google Maps/metadata-sitios/10.json',
 'D:/PROYECTO FINAL/Google Maps/metadata-sitios/11.json',
 'D:/PROYECTO FINAL/Google Maps/metadata-sitios/2.json',
 'D:/PROYECTO FINAL/Google Maps/metadata-sitios/3.json',
 'D:/PROYECTO FINAL/Google Maps/metadata-sitios/4.json',
 'D:/PROYECTO FINAL/Google Maps/metadata-sitios/5.json',
 'D:/PROYECTO FINAL/Google Maps/metadata-sitios/6.json',
 'D:/PROYECTO FINAL/Google Maps/metadata-sitios/7.json',
 'D:/PROYECTO FINAL/Google Maps/metadata-sitios/8.json',
 'D:/PROYECTO FINAL/Google Maps/metadata-sitios/9.json']

In [3]:
# Cargar todas los archivos Json y los coloca en una lista
lista_df_sitios = []
for archivo in archivos_json:       
    df = pd.read_json(archivo, lines=True)
    lista_df_sitios.append(df)

In [311]:
df_sitios = pd.concat(lista_df_sitios, ignore_index=True)     # Concatenar los Dataframes cargados en la lista

In [286]:
df_sitios.head(3)

,name,address,gmap_id,description,latitude,longitude,category,avg_rating,num_of_reviews,price,hours,MISC,state,relative_results,url
0,Porter Pharmacy,"Porter Pharmacy, 129 N Second St, Cochran, GA ...",0x88f16e41928ff687:0x883dad4fd048e8f8,None,32.388300,-83.35710,[Pharmacy],4.9,16,None,"[[Friday, 8AM–6PM], [Saturday, 8AM–12PM], [Sun...","{'Service options': ['In-store shopping', 'Sam...",Open ⋅ Closes 6PM,"[0x88f16e41929435cf:0x5b2532a2885e9ef6, 0x88f1...",https://www.google.com/maps/place//data=!4m2!3...
1,City Textile,"City Textile, 3001 E Pico Blvd, Los Angeles, C...",0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,None,34.018891,-118.21529,[Textile exporter],4.5,6,None,None,None,Open now,"[0x80c2c624136ea88b:0xb0315367ed448771, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...
2,San Soo Dang,"San Soo Dang, 761 S Vermont Ave, Los Angeles, ...",0x80c2c778e3b73d33:0xbdc58662a4a97d49,None,34.058092,-118.29213,[Korean restaurant],4.4,18,None,"[[Thursday, 6:30AM–6PM], [Friday, 6:30AM–6PM],...","{'Service options': ['Takeout', 'Dine-in', 'De...",Open ⋅ Closes 6PM,"[0x80c2c78249aba68f:0x35bf16ce61be751d, 0x80c2...",https://www.google.com/maps/place//data=!4m2!3...


In [312]:
df_sitios.isnull().sum()

name                     37
address               80511
gmap_id                   0
description         2770722
latitude                  0
longitude                 0
category              17419
avg_rating                0
num_of_reviews            0
price               2749808
hours                787405
MISC                 690834
state                746455
relative_results     295058
url                       0
dtype: int64

In [313]:
df_sitios= df_sitios.dropna(subset=['address']) # Elimina filas con address nulas
df_sitios= df_sitios.dropna(subset=['category']) # Elimina filas con category nulas

### **2. Normalizacion de datos**

#### 2.1. Se crea una tabla de "categorias" realacionada por 'gmap_id' 

In [7]:
df_category= df_sitios[['gmap_id','category']]
df_category= df_category.explode('category')

In [8]:
df_category.head(3)

,gmap_id,category
0,0x88f16e41928ff687:0x883dad4fd048e8f8,Pharmacy
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,Textile exporter
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,Korean restaurant


#### 2.2. Se crea una tabla de "miscelaneos" relacionada por 'gmap_id' 

In [9]:
# Se guardan las columnas MISC y el gmap_id en un df
df_misc= df_sitios[['gmap_id','MISC']]
df_misc= df_misc.dropna(subset=['MISC']) # Elimina filas con MISC nulo

In [10]:
# Funcion para cargar los json a una serie de pandas
def expandir_diccionario(diccionario):
    return pd.Series(diccionario)

# Se concatenas las series al df original
df_expandido = pd.concat([df_misc, df_misc['MISC'].apply(expandir_diccionario)], axis=1)
df_expandido.drop('MISC', axis=1, inplace=True)     # Se elimina la columna de MISC

In [11]:
df_misc= df_expandido
df_misc.head(3)

,gmap_id,Service options,Health & safety,Accessibility,Planning,Offerings,Amenities,Atmosphere,Payments,Popular for,Dining options,Crowd,From the business,Highlights,Recycling,Getting here,Activities,Lodging options,Health and safety
0,0x88f16e41928ff687:0x883dad4fd048e8f8,"[In-store shopping, Same-day delivery]","[Mask required, Staff required to disinfect su...",[Wheelchair accessible entrance],[Quick visit],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,"[Takeout, Dine-in, Delivery]",NaN,[Wheelchair accessible entrance],NaN,[Comfort food],[Good for kids],[Casual],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0x80c2c89923b27a41:0x32041559418d447,[In-store shopping],NaN,NaN,NaN,NaN,NaN,NaN,"[Checks, Debit cards, Credit cards]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Se revisan el contenido de las columnas para ver cuales nos sirven para el analisis

In [12]:
df_misc['Service options'].astype(str).value_counts()[:10]

Service options
nan                                                            1428216
['In-store shopping', 'Delivery']                               133839
['In-store shopping']                                            94979
['Delivery']                                                     84921
['Takeout', 'Delivery']                                          35087
['Online care']                                                  34323
['In-store pickup', 'In-store shopping', 'Delivery']             32680
['Takeout', 'Dine-in', 'Delivery']                               25223
['In-store pickup', 'In-store shopping']                         24731
['Curbside pickup', 'In-store pickup', 'In-store shopping']      24078
Name: count, dtype: int64

In [13]:
df_misc['Planning'].astype(str).value_counts()[:10]

Planning
nan                                                                              1546466
['Appointments recommended']                                                      379364
['Quick visit']                                                                   369230
['Accepts reservations']                                                           27448
['Membership required']                                                             5572
['Dinner reservations recommended', 'Accepts reservations']                         2773
['Usually a wait']                                                                  1688
['Appointments recommended', 'Quick visit']                                          576
['Accepts reservations', 'Usually a wait']                                           261
['Dinner reservations recommended', 'Accepts reservations', 'Usually a wait']        258
Name: count, dtype: int64

Observaciones:

*Al revisar todas las columnas podemos ver que solo las columnas 'Service options' y 'Planning' nos podrian servir para el analisis. Ser crearan tablas con estas columnas*

##### 2.2.1. Se crea 2 tablas 'Service options' y 'Planning' relacionada por 'gmap_id' 

In [24]:
df_Service_options= df_misc[['gmap_id','Service options']]      # Se crea un df solo las columnas gmap_id y Service Options
df_Service_options= df_Service_options.rename(columns={'Service options': 'service_option'}) # cambiar nombre de la columna
df_Service_options= df_Service_options.dropna(subset=['service_option']) # Elimina filas con columna 'Service Opciones nulas'
df_Service_options= df_Service_options.explode('service_option')

In [25]:
df_Service_options.head(3)

,gmap_id,service_option
0,0x88f16e41928ff687:0x883dad4fd048e8f8,In-store shopping
0,0x88f16e41928ff687:0x883dad4fd048e8f8,Same-day delivery
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,Takeout


In [28]:
df_Planning= df_misc[['gmap_id','Planning']]      # Se crea un df solo las columnas gmap_id y Planning
df_Planning= df_Planning.rename(columns={'Planning': 'planning_option'}) # cambiar nombre de la columna
df_Planning= df_Planning.dropna(subset=['planning_option']) # Elimina filas con columna 'Service Opciones nulas'
df_Planning= df_Planning.explode('planning_option')

In [29]:
df_Planning.head(3)

,gmap_id,planning_option
0,0x88f16e41928ff687:0x883dad4fd048e8f8,Quick visit
10,0x87ec2394c2cd9d2d:0xd1119cfbee0da6f3,Appointments recommended
11,0x87ee95d1e474531b:0xb432dd96285cdc6b,Appointments recommended


#### 2.3. Se extrae la Ciudad y Estado de la columna 'address'

In [314]:
# En pruebas se detecto que esta fila estaba dando problemas con la funcion para extraer, asi que se procede a eliminarla
df_sitios= df_sitios[df_sitios['address'] != "〒10028 New York, Lexington Ave, (New) Ichie Japanese Restaurant"]

In [315]:
# Funcion para extraer la La ciudad y el estrado de la direccion
def Ext_Ciudad_Estado(dir):
    ciudad= "SIN DATO"
    estado= "SIN DATO"     
    if len(str(dir)) > 10:  
        lista= str(dir).split(',')
        if len(lista) > 2:
            codigo= lista[-1][1:3]
            df_filtro= df_estados_USA[df_estados_USA['nombre_corto'].str.contains(codigo)]        
            if not df_filtro.empty:            
                ciudad = lista[-2].strip()
                estado= df_filtro.nombre_largo.values[0].strip()
    return ciudad, estado

In [316]:
df_sitios[['ciudad','estado']] = df_sitios.apply(lambda x: Ext_Ciudad_Estado(x['address']), axis=1, result_type='expand')

#### 2.4. Se borran las columnas innecesarias

In [318]:
#Borrar Columnas
df_sitios= df_sitios.drop(['relative_results','address', 'num_of_reviews', 'description', 'url','category', 'MISC', 'hours'], axis=1) 
# Ordena el orden de las columnas
df_sitios= df_sitios[['gmap_id','name', 'ciudad', 'estado', 'latitude', 'longitude', 'avg_rating', 'price', 'state']]    

In [322]:
df_sitios.head(3)

,gmap_id,name,ciudad,estado,latitude,longitude,avg_rating,price,state
0,0x88f16e41928ff687:0x883dad4fd048e8f8,Porter Pharmacy,Cochran,Georgia,32.388300,-83.35710,4.9,None,Open ⋅ Closes 6PM
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,City Textile,Los Angeles,California,34.018891,-118.21529,4.5,None,Open now
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,San Soo Dang,Los Angeles,California,34.058092,-118.29213,4.4,None,Open ⋅ Closes 6PM


In [323]:
df_sitios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2927085 entries, 0 to 3025010
Data columns (total 9 columns):
 #   Column      Dtype  
---  ------      -----  
 0   gmap_id     object 
 1   name        object 
 2   ciudad      object 
 3   estado      object 
 4   latitude    float64
 5   longitude   float64
 6   avg_rating  float64
 7   price       object 
 8   state       object 
dtypes: float64(3), object(6)
memory usage: 223.3+ MB


### **3. Exportacion de DataFrames a archivos .parquet**

In [324]:
df_sitios.to_parquet('D:/PROYECTO FINAL/Google Maps/Sitios_GMaps.parquet', index=False)

In [ ]:
df_category.to_parquet('D:/PROYECTO FINAL/Google Maps/Category_Sitios.parquet', index=False)

In [32]:
df_Service_options.to_parquet('D:/PROYECTO FINAL/Google Maps/Service-options_Sitios.parquet', index=False)

In [33]:
df_Planning.to_parquet('D:/PROYECTO FINAL/Google Maps/Planning-options_Sitios.parquet', index=False)

#########################################################################################################################

### Reviews Estados

In [2]:
rutaJson = "D:/PROYECTO FINAL/Google Maps/reviews-estados/review-Arizona/"     # Ruta de los json de Sitios

In [3]:
archivos_json = [os.path.join(rutaJson, archivo) for archivo in os.listdir(rutaJson) if archivo.endswith('.json')]
archivos_json

['D:/PROYECTO FINAL/Google Maps/reviews-estados/review-Arizona/1.json',
 'D:/PROYECTO FINAL/Google Maps/reviews-estados/review-Arizona/10.json',
 'D:/PROYECTO FINAL/Google Maps/reviews-estados/review-Arizona/11.json',
 'D:/PROYECTO FINAL/Google Maps/reviews-estados/review-Arizona/12.json',
 'D:/PROYECTO FINAL/Google Maps/reviews-estados/review-Arizona/13.json',
 'D:/PROYECTO FINAL/Google Maps/reviews-estados/review-Arizona/14.json',
 'D:/PROYECTO FINAL/Google Maps/reviews-estados/review-Arizona/2.json',
 'D:/PROYECTO FINAL/Google Maps/reviews-estados/review-Arizona/3.json',
 'D:/PROYECTO FINAL/Google Maps/reviews-estados/review-Arizona/4.json',
 'D:/PROYECTO FINAL/Google Maps/reviews-estados/review-Arizona/5.json',
 'D:/PROYECTO FINAL/Google Maps/reviews-estados/review-Arizona/6.json',
 'D:/PROYECTO FINAL/Google Maps/reviews-estados/review-Arizona/7.json',
 'D:/PROYECTO FINAL/Google Maps/reviews-estados/review-Arizona/8.json',
 'D:/PROYECTO FINAL/Google Maps/reviews-estados/review-Ariz

In [4]:
# Cargar todas los archivos Json y colocarlos en una lista
lista_df_review = []
for archivo in archivos_json:       
    df = pd.read_json(archivo, lines=True)
    lista_df_review.append(df)

In [5]:
df_review_Arizona = pd.concat(lista_df_review, ignore_index=True)     # Concatenar los Dataframes cargados en la lista

In [6]:
df_review_Arizona.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100000 entries, 0 to 2099999
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 128.2+ MB


In [7]:
df_review_Arizona.head(5)

,user_id,name,time,rating,text,pics,resp,gmap_id
0,1.183560e+20,Vicki Ullery,1628718071358,5,Gwen was so informative and made me feel comfo...,None,None,0x872b5dae3c5fcfa7:0xf0f8b286a2537821
1,1.126523e+20,Doc Kovacs,1609796129431,5,Dr. Huyhn is an ethical skilled dentist. Her s...,None,None,0x872b5dae3c5fcfa7:0xf0f8b286a2537821
2,1.089879e+20,Pedro Diaz,1594225668169,1,Not sure if one star is fair but my situation ...,None,None,0x872b5dae3c5fcfa7:0xf0f8b286a2537821
3,1.069277e+20,Brandon Ray,1622676189509,5,Best dentist office I've ever had. Love everyt...,None,None,0x872b5dae3c5fcfa7:0xf0f8b286a2537821
4,1.090535e+20,T. Bennett,1434829263041,5,I've been coming to Oasis dentistry for a whil...,None,None,0x872b5dae3c5fcfa7:0xf0f8b286a2537821


In [8]:
df_review_Arizona.isnull().sum()

user_id          0
name             0
time             0
rating           0
text        832184
pics       2036129
resp       1760851
gmap_id          0
dtype: int64

In [9]:
df_review_Arizona= df_review_Arizona.drop('pics', axis=1)       #Elimina la columna pics

In [19]:
df_review_Arizona['time'] = pd.to_datetime(df_review_Arizona['time'],unit='ms')     #Transforma la columna de tipo timestamp a formato datetime

In [20]:
df_review_Arizona

,user_id,name,time,rating,text,resp,gmap_id
0,1.183560e+20,Vicki Ullery,2021-08-11 21:41:11.358,5,Gwen was so informative and made me feel comfo...,None,0x872b5dae3c5fcfa7:0xf0f8b286a2537821
1,1.126523e+20,Doc Kovacs,2021-01-04 21:35:29.431,5,Dr. Huyhn is an ethical skilled dentist. Her s...,None,0x872b5dae3c5fcfa7:0xf0f8b286a2537821
2,1.089879e+20,Pedro Diaz,2020-07-08 16:27:48.169,1,Not sure if one star is fair but my situation ...,None,0x872b5dae3c5fcfa7:0xf0f8b286a2537821
3,1.069277e+20,Brandon Ray,2021-06-02 23:23:09.509,5,Best dentist office I've ever had. Love everyt...,None,0x872b5dae3c5fcfa7:0xf0f8b286a2537821
4,1.090535e+20,T. Bennett,2015-06-20 19:41:03.041,5,I've been coming to Oasis dentistry for a whil...,None,0x872b5dae3c5fcfa7:0xf0f8b286a2537821
...,...,...,...,...,...,...,...
2099995,1.107824e+20,Nina Kitsmiller,2018-01-19 09:52:37.532,4,Cindy is great.... staff is ok.,None,0x872a3e98eb27210f:0xa7a246259f678edf
2099996,1.053941e+20,Green Leaf,2019-04-21 01:49:56.904,3,None,None,0x872a3e98eb27210f:0xa7a246259f678edf
2099997,1.006415e+20,Krystal Scoggins,2019-10-09 11:49:21.138,5,None,None,0x872a3e98eb27210f:0xa7a246259f678edf
2099998,1.172314e+20,John DiMarco,2019-01-11 21:54:37.304,5,None,None,0x872a3e98eb27210f:0xa7a246259f678edf


In [36]:
df_review_Arizona.to_parquet('D:/PROYECTO FINAL/Google Maps/Review_Arizona.parquet', index=False)